<a href="https://colab.research.google.com/github/tonystz/gitpod/blob/main/strings_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [ ]:
import numpy as np
ttt = np.asarray([ "stuff" + str(i)  for i in range(0,2) ])

print( ttt.dtype, type(ttt[0]) ) 
print(ttt)

s= np.array(['abcdf'], dtype=object)
print( s.dtype, type(s[0]) )

#print("ctype:",np.ctypeslib.as_ctypes_type(s.dtype))

<U6 <class 'numpy.str_'>
['stuff0' 'stuff1']
object <class 'str'>


## string test

In [32]:
%%writefile s2.py
import time
import numpy as np
from pycuda import driver, compiler, gpuarray, tools
import math
from sys import getsizeof

import pycuda.autoinit

kernel_code1 = """
__global__ void test1(char** d_wordList) {
    (d_wordList[blockIdx.x][threadIdx.x])++;      
}
    """

kernel_code2 = """
__global__ void test2(char* d_wordList, size_t *offsets) {

    int idx = threadIdx.x+blockDim.x*blockIdx.x;
    printf("index: threadIdx.x=%d blockIdx.x=%d  blockDim.x=%d \\n",threadIdx.x,blockIdx.x,blockDim.x);
    printf("Hello from thread %d, my string is %s ,offsets:%u \\n", idx, d_wordList+offsets[idx],offsets[blockIdx.x]);
    (d_wordList[offsets[blockIdx.x] + threadIdx.x])++;
}
    """




mod = compiler.SourceModule(kernel_code1)
ker_test1 = mod.get_function("test1")



wordList = ['asd','bsd','csd']

d_words = []

for word in wordList:
    d_words.append(gpuarray.to_gpu(np.array(word, dtype=str)))

d_wordList = gpuarray.to_gpu(np.array([word.ptr for word in d_words], dtype=np.uintp))

ker_test1(d_wordList, block=(3,1,1), grid=(3,1,1))

for word in d_words:
  result = word.get()
  print(result)

mod2 = compiler.SourceModule(kernel_code2)
ker_test2 = mod2.get_function("test2")

d_words2 = gpuarray.to_gpu(np.array(['asd','bsd','csd'], dtype=np.string_))
offsets = gpuarray.to_gpu(np.array([0,3,6,9], dtype=np.uint64))
ker_test2(d_words2, offsets, block=(3,1,1), grid=(1,1,1))
h_words2 = d_words2.get()
print(h_words2)

Overwriting s2.py


In [33]:
!python s2.py

𐅢sd
𐅣sd
𐅤sd
index: threadIdx.x=0 blockIdx.x=0  blockDim.x=3 
index: threadIdx.x=1 blockIdx.x=0  blockDim.x=3 
index: threadIdx.x=2 blockIdx.x=0  blockDim.x=3 
Hello from thread 0, my string is asdbsdcsd ,offsets:0 
Hello from thread 1, my string is bsdcsd ,offsets:0 
Hello from thread 2, my string is csd ,offsets:0 
[b'bte' b'bsd' b'csd']


## all string size must equal?

In [2]:
%%writefile s.py
#!python 
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda import gpuarray
import numpy as np

mod = SourceModule("""
    #include <stdio.h>
    const int strLen=6;
    __global__ void say_hi(char*out_gpu, unsigned int *s_offset_gpu)
    { 
      int idx = threadIdx.x + blockIdx.x * blockDim.x;

      //printf("index: threadIdx.x=%d blockIdx.x=%d  blockDim.x=%d \\n",threadIdx.x,blockIdx.x,blockDim.x);
      //printf("thread id: [%d]  offset:%d \\n",idx,s_offset_gpu[idx]);
      printf("thread id: [%d]  offset:%d  get str:%s  strlen=%d \\n",idx,s_offset_gpu[idx],out_gpu+s_offset_gpu[idx]+1, strLen);

      // string len is 6
      char sub[strLen+1];
      for(int i = 0; i<strLen; i++)
      {
        sub[i]=out_gpu[idx*strLen+i];
      }
      printf("substring:%s\\n",sub);
    }
    """)

say_hi = mod.get_function("say_hi")

#padding string with \0 char
s= np.array(['abcmh\0','edfg\0\0','s12wc\0','h45q\0\0'], dtype=np.string_)

offset=[0]
for i in s:
  offset.append(offset[-1]+len(i))
s_offset=np.array(offset,dtype=np.uint32)
print('shape:',s.shape)
print(s,s.data)
print('offset:',s_offset,s_offset.shape)

s_gpu = gpuarray.to_gpu(s)
s_offset_gpu = gpuarray.to_gpu(s_offset)

say_hi(s_gpu,s_offset_gpu,block=(s.size,1,1),grid=(1,1,1))
print('modify:',s_gpu.get(),s_offset_gpu.get())

print("s_offset ctype:",np.ctypeslib.as_ctypes_type(s_offset.dtype))

Writing s.py


In [3]:
!python s.py

shape: (4,)
[b'abcmh' b'edfg' b's12wc' b'h45q'] <memory at 0x7f4a8493d580>
offset: [ 0  5  9 14 18] (5,)
thread id: [0]  offset:0  get str:bcmh  strlen=6 
thread id: [1]  offset:5  get str:edfg  strlen=6 
thread id: [2]  offset:9  get str:  strlen=6 
thread id: [3]  offset:14  get str:wc  strlen=6 
substring:abcmh
substring:edfg
substring:s12wc
substring:h45q
modify: [b'abcmh' b'edfg' b's12wc' b'h45q'] [ 0  5  9 14 18]
s_offset ctype: <class 'ctypes.c_uint'>


## test un-equal string process?

In [16]:
%%writefile su.py
#!python 
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda import gpuarray
import numpy as np

mod = SourceModule("""
    #include <stdio.h>

    __global__ void say_hi(char*out_gpu, unsigned int *s_offset_gpu)
    {  //max string len is 13
      const int strLen=13;
      int idx = threadIdx.x + blockIdx.x * blockDim.x;
      //printf("index: threadIdx.x=%d blockIdx.x=%d  blockDim.x=%d \\n",threadIdx.x,blockIdx.x,blockDim.x);
      //printf("thread id: [%d]  offset:%d \\n",idx,s_offset_gpu[idx]);
      printf("thread id: [%d]  offset:%d  get str:%s\\n",idx,s_offset_gpu[idx],out_gpu+s_offset_gpu[idx]);
  
      char sub[strLen+1];
      for(int i = 0; i<strLen; i++)
      {
        sub[i]=out_gpu[idx*strLen+i];
      }
      printf("thread[%d]:substring:%s\\n",idx,sub);
    }
    """)

say_hi = mod.get_function("say_hi")


s= np.array(['abc\0','edfg\0','stz12345678b\0'], dtype=np.string_)

offset=[0]
for i in s:
  offset.append(offset[-1]+len(i))
s_offset=np.array(offset,dtype=np.uint32)
print('shape:',s.shape)
print(s,s.data)
print('offset:',s_offset,s_offset.shape)

s_gpu = gpuarray.to_gpu(s)
s_offset_gpu = gpuarray.to_gpu(s_offset)

say_hi(s_gpu,s_offset_gpu,block=(s.size,1,1),grid=(1,1,1))
print('modify:',s_gpu.get(),s_offset_gpu.get())

print("s_offset ctype:",np.ctypeslib.as_ctypes_type(s_offset.dtype))

Overwriting su.py


In [17]:
!python su.py

shape: (3,)
[b'abc' b'edfg' b'stz12345678b'] <memory at 0x7f01c3dbb580>
offset: [ 0  3  7 19] (4,)
thread id: [0]  offset:0  get str:abc
thread id: [1]  offset:3  get str:
thread id: [2]  offset:7  get str:
thread[0]:substring:abc
thread[1]:substring:edfg
thread[2]:substring:stz12345678b
modify: [b'abc' b'edfg' b'stz12345678b'] [ 0  3  7 19]
s_offset ctype: <class 'ctypes.c_uint'>


In [19]:
%%writefile t389.cu
#include <stdio.h>
#include <string.h>

#define nTPB 256

__global__ void kern_1D(char *data, unsigned *indices, unsigned num_strings){

  int idx = threadIdx.x+blockDim.x*blockIdx.x;
  if (idx < num_strings)
    printf("Hello from thread %d, offset=%d, my string is %s\n", idx, indices[idx],data+indices[idx]);
}

__global__ void kern_2D(char **data, unsigned num_strings){

  int idx = threadIdx.x+blockDim.x*blockIdx.x;
  if (idx < num_strings)
    printf("Hello from thread %d, my string is %s\n", idx, data[idx]);
}

int main(){

  const int num_strings = 3;
  const char s0[] = "s1\0";
  const char s1[] = "s2\0";
  const char s2[] = "sstz3";
  int ds[num_strings];
  ds[0] = sizeof(s0)/sizeof(char);
  ds[1] = sizeof(s1)/sizeof(char);
  ds[2] = sizeof(s2)/sizeof(char);
  // pretend we have a dynamically allocated char** array
  char **data;
  data = (char **)malloc(num_strings*sizeof(char *));
  data[0] = (char *)malloc(ds[0]*sizeof(char));
  data[1] = (char *)malloc(ds[1]*sizeof(char));
  data[2] = (char *)malloc(ds[2]*sizeof(char));
  // initialize said array
  strcpy(data[0], s0);
  strcpy(data[1], s1);
  strcpy(data[2], s2);
  // method 1: "flattening"
  char *fdata = (char *)malloc((ds[0]+ds[1]+ds[2])*sizeof(char));
  unsigned *ind   = (unsigned *)malloc(num_strings*sizeof(unsigned));
  unsigned next = 0;
  for (int i = 0; i < num_strings; i++){
    strcpy(fdata+next, data[i]);
    ind[i] = next;
    next += ds[i];}
  //copy to device
  char *d_fdata;
  unsigned *d_ind;
  cudaMalloc(&d_fdata, next*sizeof(char));
  cudaMalloc(&d_ind, num_strings*sizeof(unsigned));
  cudaMemcpy(d_fdata, fdata, next*sizeof(char), cudaMemcpyHostToDevice);
  cudaMemcpy(d_ind, ind, num_strings*sizeof(unsigned), cudaMemcpyHostToDevice);
  printf("method 1: %d  --> %d\n",(num_strings+nTPB-1)/nTPB,nTPB);
  kern_1D<<<(num_strings+nTPB-1)/nTPB, nTPB>>>(d_fdata, d_ind, num_strings);
  cudaDeviceSynchronize();
  //method 2: "2D" (pointer-to-pointer) array
  char **d_data;
  cudaMalloc(&d_data, num_strings*sizeof(char *));
  char **d_temp_data;
  d_temp_data = (char **)malloc(num_strings*sizeof(char *));
  for (int i = 0; i < num_strings; i++){
    cudaMalloc(&(d_temp_data[i]), ds[i]*sizeof(char));
    cudaMemcpy(d_temp_data[i], data[i], ds[i]*sizeof(char), cudaMemcpyHostToDevice);
    cudaMemcpy(d_data+i, &(d_temp_data[i]), sizeof(char *), cudaMemcpyHostToDevice);}
  printf("method 2:\n");
  kern_2D<<<(num_strings+nTPB-1)/nTPB, nTPB>>>(d_data, num_strings);
  cudaDeviceSynchronize();
  // method 3: managed allocations
  // start over with a managed char** array
  char **m_data;
  cudaMallocManaged(&m_data, num_strings*sizeof(char *));
  cudaMallocManaged(&(m_data[0]), ds[0]*sizeof(char));
  cudaMallocManaged(&(m_data[1]), ds[1]*sizeof(char));
  cudaMallocManaged(&(m_data[2]), ds[2]*sizeof(char));
  // initialize said array
  strcpy(m_data[0], s0);
  strcpy(m_data[1], s1);
  strcpy(m_data[2], s2);
  // call kernel directly on managed data
  printf("method 3:\n");
  kern_2D<<<(num_strings+nTPB-1)/nTPB, nTPB>>>(m_data, num_strings);
  cudaDeviceSynchronize();

  return 0;
}

Overwriting t389.cu


In [20]:
!nvcc t389.cu -o t389
!nvprof ./t389

==4961== NVPROF is profiling process 4961, command: ./t389
method 1: 1  --> 256
Hello from thread 0, offset=0, my string is s1
Hello from thread 1, offset=4, my string is s2
Hello from thread 2, offset=8, my string is sstz3
method 2:
Hello from thread 0, my string is s1
Hello from thread 1, my string is s2
Hello from thread 2, my string is sstz3
method 3:
Hello from thread 0, my string is s1
Hello from thread 1, my string is s2
Hello from thread 2, my string is sstz3
==4961== Profiling application: ./t389
==4961== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   82.09%  591.70us         2  295.85us  98.494us  493.20us  kern_2D(char**, unsigned int)
                   16.23%  116.99us         1  116.99us  116.99us  116.99us  kern_1D(char*, unsigned int*, unsigned int)
                    1.67%  12.062us         8  1.5070us  1.4080us  2.0160us  [CUDA memcpy HtoD]
      API calls:   93.68%  338.21ms         6  56.368ms 

In [25]:
!nvcc -o t389 t389.cu
!compute-sanitizer ./t389

========= COMPUTE-SANITIZER
method 1: 1  --> 256
Hello from thread 0, offset=0, my string is s1
Hello from thread 1, offset=4, my string is s2
Hello from thread 2, offset=8, my string is sstz3
method 2:
Hello from thread 0, my string is s1
Hello from thread 1, my string is s2
Hello from thread 2, my string is sstz3
method 3:
Hello from thread 0, my string is s1
Hello from thread 1, my string is s2
Hello from thread 2, my string is sstz3
========= ERROR SUMMARY: 0 errors


## test struct with number
https://github.com/inducer/pycuda/blob/main/examples/demo_struct.py

In [28]:
%%writefile demo_struct.py
# prepared invocations and structures -----------------------------------------
import pycuda.driver as cuda
import pycuda.autoinit
import numpy
import struct
from pycuda.compiler import SourceModule

class DoubleOpStruct:
    mem_size = 8 + numpy.uintp(0).nbytes
    def __init__(self, array, struct_arr_ptr):
        self.data = cuda.to_device(array)
        self.shape, self.dtype = array.shape, array.dtype

        packed_args = struct.pack("ixP", array.size, numpy.uintp(self.data))
        cuda.memcpy_htod(struct_arr_ptr, packed_args)

    def __str__(self):
        return str(cuda.from_device(self.data, self.shape, self.dtype))

struct_arr = cuda.mem_alloc(2 * DoubleOpStruct.mem_size)
do2_ptr = int(struct_arr) + DoubleOpStruct.mem_size

array1 = DoubleOpStruct(numpy.array([1, 2, 3], dtype=numpy.float32), struct_arr)
array2 = DoubleOpStruct(numpy.array([0, 4], dtype=numpy.float32), do2_ptr)

print("original arrays")
print(array1)
print(array2)

mod = SourceModule("""
    struct DoubleOperation {
        int datalen, __padding; // so 64-bit ptrs can be aligned
        float *ptr;
    };


    __global__ void double_array(DoubleOperation *a)
    {
        a = a + blockIdx.x;
        for (int idx = threadIdx.x; idx < a->datalen; idx += blockDim.x)
        {
            float *a_ptr = a->ptr;
            a_ptr[idx] *= 2;
        }
    }
    """)
func = mod.get_function("double_array")
func(struct_arr, block=(32, 1, 1), grid=(2, 1))

print("doubled arrays")
print(array1)
print(array2)

func(numpy.uintp(do2_ptr), block=(32, 1, 1), grid=(1, 1))
print("doubled second only")
print(array1)
print(array2)

if cuda.get_version() < (4, ):
    func.prepare("P", block=(32, 1, 1))
    func.prepared_call((2, 1), struct_arr)
else:
    func.prepare("P")
    block = (32, 1, 1)
    func.prepared_call((2, 1), block, struct_arr)


print("doubled again")
print(array1)
print(array2)

if cuda.get_version() < (4, ):
    func.prepared_call((1, 1), do2_ptr)
else:
    func.prepared_call((1, 1), block, do2_ptr)


print("doubled second only again")
print(array1)
print(array2)


Overwriting demo_struct.py


In [29]:
!python demo_struct.py

original arrays
[1. 2. 3.]
[0. 4.]
doubled arrays
[2. 4. 6.]
[0. 8.]
doubled second only
[2. 4. 6.]
[ 0. 16.]
doubled again
[ 4.  8. 12.]
[ 0. 32.]
doubled second only again
[ 4.  8. 12.]
[ 0. 64.]


## test struct with string

In [27]:
!python s2.py

𐅢sd
𐅣sd
𐅤sd
/content/s2.py:48: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(7): warning #181-D: argument is incompatible with corresponding format string conversion


  mod2 = compiler.SourceModule(kernel_code2)
index: threadIdx.x=0 blockIdx.x=0  blockDim.x=3 
index: threadIdx.x=1 blockIdx.x=0  blockDim.x=3 
index: threadIdx.x=2 blockIdx.x=0  blockDim.x=3 
Hello from thread 0, my string is asdbsdcsd ,offsets:0 
Hello from thread 1, my string is bsdcsd ,offsets:0 
Hello from thread 2, my string is csd ,offsets:0 
[b'bte' b'bsd' b'csd']


### reference:
https://stackoverflow.com/questions/34700999/cuda-passing-char-to-kernel/34712905
https://documen.tician.de/pycuda/tutorial.html#structures
https://stackoverflow.com/questions/48038577/pycuda-numpy-and-working-with-strings-in-general